In [1]:
#importing all te req lib
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing import image
import numpy as np

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 28,28

#defining the data directories
train_data_dir= 'tumor_detect/train'
validation_data_dir= 'tumor_detect/validation'
n_training_sample= 100
n_validation_sample= 50
epochs=20
batch_size=10

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
#defining the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(), 
  tf.keras.layers.Dense(128, activation=tf.nn.relu), 
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)])

In [5]:
#defining the optimizer and metrics
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=n_training_sample // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=n_validation_sample // batch_size)

Found 144 images belonging to 2 classes.
Found 52 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
10/10 [==============================] - 0s 45ms/step - loss: 0.6796 - accuracy: 0.6064 - val_loss: 0.5796 - val_accuracy: 0.7800
Epoch 2/20
10/10 [==============================] - 0s 43ms/step - loss: 0.6139 - accuracy: 0.6702 - val_loss: 0.5283 - val_accuracy: 0.7200
Epoch 3/20
10/10 [==============================] - 0s 45ms/step - loss: 0.6492 - accuracy: 0.6064 - val_loss: 0.5878 - val_accuracy: 0.6800
Epoch 4/20
10/10 [==============================] - 0s 33ms/step - loss: 0.5890 - accuracy: 0.6900 - val_loss: 0.5122 - val_accuracy: 0.7600
Epoch 5/20
10/10 [==============================] - 0s 32ms/step - loss: 0.5423 - accuracy: 0.7553 - val_loss: 0.4794 - val_accuracy: 0.7400
Epoch 6/20
10/10 [==============================] - 0s 41ms/step - loss: 0.5453 - accuracy: 0.7234 - val_loss: 0.5388 - val_accuracy: 0.74

In [30]:
#testing the model
pred= image.load_img('tumor_detect/test/Y3.jpg', target_size=(28,28))
pred=image.img_to_array(pred)
pred= np.expand_dims(pred, axis=0)
result= model.predict(pred)
print(result)

[[0. 1.]]


In [31]:
if result[0][1]==1:
    answer='Its a Tumor'
else:
    answer='There is no tumor'
print(answer)

Its a Tumor
